# Data Preprocessing Pipeline

**Swiss Energy Portfolio Optimization - Modular Data Pipeline**

This notebook provides a software-engineered approach to data preprocessing for energy portfolio optimization.

---

## Architecture

```
dumping/          # Raw data files (downloaded externally)
  ├── solar_incidence/
  ├── wind_incidence/
  ├── energy_price_all_2024.json
  ├── chf_to_eur_2024.csv
  ├── ...
  
import/           # Processed data ready for optimization
  ├── solar_incidence_hourly_2024.csv
  ├── wind_incidence_hourly_2024.csv
  ├── spot_price_hourly.csv
  ├── ...
```

---

## Pipeline Stages

| Stage | Input | Output | Description |
|-------|-------|--------|-------------|
| **1. Solar** | GRIB files | solar_incidence_hourly_2024.csv | GHI per location |
| **2. Wind** | GRIB files | wind_incidence_hourly_2024.csv | Wind speed per location |
| **3. Prices** | JSON + CSV | spot_price_hourly.csv | Spot prices in CHF/MWh |
| **4. Demand** | CSV | monthly_hourly_load_values_2024.csv | Electricity demand |
| **5. Water** | Monthly CSV | water_quarterly_ror_2024.csv | Hydro generation |
| **6. PPU** | Excel + Python | ppu_efficiency_lcoe_analysis.csv | Cost analysis |

---

## Instructions

1. **Download raw data** into `dumping/` folder (see data sources below)
2. **Run this notebook** from top to bottom
3. **Processed files** will be saved to `import/` folder
4. **Run `Energy_Portfolio_Optimization.ipynb`** for portfolio analysis


## 0. Configuration & Imports

All dependencies and configuration in one place following best practices.


In [ ]:
# ============================================================================
# IMPORTS & CONFIGURATION
# ============================================================================

# Standard library
import os
import sys
import json
import csv
import warnings
from pathlib import Path
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Union

# Data handling
import numpy as np
import pandas as pd

# Scientific computing
from scipy.interpolate import CubicSpline
from scipy import stats

# Visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.patches import Patch
import seaborn as sns

# Geospatial data (for GRIB processing)
try:
    import xarray as xr
    HAS_XARRAY = True
except ImportError:
    HAS_XARRAY = False
    print("⚠️ xarray not installed. GRIB processing will be skipped.")
    print("   Install with: pip install xarray cfgrib")

# Suppress warnings
warnings.filterwarnings('ignore')

# ============================================================================
# MATPLOTLIB CONFIGURATION
# ============================================================================

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

# ============================================================================
# PATH CONFIGURATION
# ============================================================================

# Directories
DUMPING_DIR = Path('dumping')  # Raw data
IMPORT_DIR = Path('import')    # Processed data
PLOT_DIR = IMPORT_DIR / 'plots'  # Visualization outputs

# Create directories
for d in [DUMPING_DIR, IMPORT_DIR, PLOT_DIR]:
    d.mkdir(exist_ok=True, parents=True)

# ============================================================================
# GLOBAL CONSTANTS
# ============================================================================

YEAR = 2024
N_HOURS_YEAR = 8784 if YEAR % 4 == 0 else 8760  # 2024 is leap year
SWISS_SURFACE_AREA_M2 = 41_285_000_000  # 41,285 km²

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def print_section(title: str, char: str = "=", width: int = 80) -> None:
    """Print a formatted section header."""
    print("\n" + char * width)
    print(title)
    print(char * width)

def print_subsection(title: str) -> None:
    """Print a formatted subsection header."""
    print(f"\n{'─' * 80}")
    print(f"  {title}")
    print(f"{'─' * 80}")

def validate_file(path: Path, description: str = "") -> bool:
    """Check if a file exists and print status with file size."""
    if path.exists():
        size_mb = path.stat().st_size / (1024 * 1024)
        print(f"  ✅ {path.name}: {size_mb:.2f} MB")
        return True
    else:
        print(f"  ❌ {path.name}: MISSING {f'- {description}' if description else ''}")
        return False

def save_plot(fig, filename: str, dpi: int = 150) -> None:
    """Save a plot to the plot directory."""
    output_path = PLOT_DIR / filename
    fig.savefig(output_path, dpi=dpi, bbox_inches='tight', facecolor='white')
    print(f"  💾 Saved plot: {output_path.name}")

def print_dataframe_info(df: pd.DataFrame, name: str) -> None:
    """Print comprehensive DataFrame information."""
    print(f"\n📊 {name}")
    print(f"  Shape: {df.shape[0]:,} rows × {df.shape[1]:,} columns")
    print(f"  Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    if hasattr(df.index, 'min') and hasattr(df.index, 'max'):
        try:
            print(f"  Index range: {df.index.min()} to {df.index.max()}")
        except:
            pass
    print(f"  Null values: {df.isnull().sum().sum():,}")

def compute_statistics(data: np.ndarray, name: str) -> Dict[str, float]:
    """Compute comprehensive statistics for a dataset."""
    clean_data = data[~np.isnan(data)]
    return {
        'count': len(clean_data),
        'mean': np.mean(clean_data),
        'median': np.median(clean_data),
        'std': np.std(clean_data),
        'min': np.min(clean_data),
        'max': np.max(clean_data),
        'q25': np.percentile(clean_data, 25),
        'q75': np.percentile(clean_data, 75),
        'skewness': stats.skew(clean_data),
        'kurtosis': stats.kurtosis(clean_data)
    }

def print_statistics(stats_dict: Dict[str, float], title: str) -> None:
    """Print formatted statistics."""
    print(f"\n📈 {title}")
    print(f"  Count:    {stats_dict['count']:,.0f}")
    print(f"  Mean:     {stats_dict['mean']:.4f}")
    print(f"  Median:   {stats_dict['median']:.4f}")
    print(f"  Std:      {stats_dict['std']:.4f}")
    print(f"  Min:      {stats_dict['min']:.4f}")
    print(f"  Max:      {stats_dict['max']:.4f}")
    print(f"  Q25:      {stats_dict['q25']:.4f}")
    print(f"  Q75:      {stats_dict['q75']:.4f}")
    print(f"  Skewness: {stats_dict['skewness']:.4f}")
    print(f"  Kurtosis: {stats_dict['kurtosis']:.4f}")

# ============================================================================
# STATUS
# ============================================================================

print_section("DATA PREPROCESSING PIPELINE - SWISS ENERGY PORTFOLIO OPTIMIZATION")
print(f"✅ All imports successful")
print(f"   Year: {YEAR} ({N_HOURS_YEAR} hours)")
print(f"   Dumping directory: {DUMPING_DIR.absolute()}")
print(f"   Import directory: {IMPORT_DIR.absolute()}")
print(f"   Plot directory: {PLOT_DIR.absolute()}")
print(f"   xarray available: {HAS_XARRAY}")


## 1. Solar Incidence Data

**Source**: ERA5 reanalysis GRIB files  
**Input**: `dumping/solar_incidence/*.grib`  
**Output**: `import/solar_incidence_hourly_2024.csv`, `import/solar_incidence_ranking.csv`  
**Processing**: Extract SSRD, convert J/m² to kWh/m², pivot to hourly × locations format


### 1.1 Data Treatment: GRIB to CSV Conversion


In [ ]:
# ============================================================================
# 1.1 SOLAR: GRIB TO CSV CONVERSION
# ============================================================================

def process_solar_grib_to_csv(input_dir: Path, output_csv: Path) -> Optional[pd.DataFrame]:
    """
    Process solar irradiance GRIB files into hourly CSV.
    
    Parameters
    ----------
    input_dir : Path
        Directory containing GRIB files
    output_csv : Path
        Output CSV path
        
    Returns
    -------
    pd.DataFrame or None
        Pivoted DataFrame with hourly solar incidence per location (hours × locations)
        Returns None if processing fails
    """
    if not HAS_XARRAY:
        print("  ⚠️ xarray not available - cannot process GRIB files")
        return None
    
    print_subsection("Processing Solar GRIB Files")
    
    # Find all GRIB files (exclude .idx index files)
    grib_files = sorted([p for p in input_dir.rglob("*") 
                         if p.is_file() and not p.name.endswith('.idx')])
    
    if not grib_files:
        print(f"  ❌ No GRIB files found in {input_dir}")
        return None
    
    print(f"  📂 Found {len(grib_files)} GRIB files")
    
    # Load all datasets
    all_dsets = []
    for f in grib_files:
        try:
            ds = xr.open_dataset(
                f,
                engine="cfgrib",
                backend_kwargs={"filter_by_keys": {"shortName": "ssrd"}},
            )
            all_dsets.append(ds)
            print(f"     ✓ Loaded {f.name}")
        except Exception as e:
            print(f"     ⚠️ Failed to open {f.name}: {e}")
    
    if not all_dsets:
        print("  ❌ No GRIB datasets could be opened")
        return None
    
    print(f"  🔄 Combining {len(all_dsets)} datasets...")
    
    # Combine all months
    ds_all = xr.combine_by_coords(all_dsets, combine_attrs="override")
    
    # Find SSRD variable (Surface Solar Radiation Downwards)
    var_name = next((v for v in ds_all.data_vars 
                     if v.lower() in ("ssrd", "surface_solar_radiation_downwards")), None)
    if var_name is None:
        var_name = next((v for v in ds_all.data_vars if "ssrd" in v.lower()), None)
    if var_name is None:
        print(f"  ❌ No SSRD variable found. Variables: {list(ds_all.data_vars)}")
        return None
    
    print(f"  ⚙️  Found variable: {var_name}")
    print(f"  ⚙️  Converting J/m² to kWh/m²...")
    
    # Convert J/m² to kWh/m²
    ghi_kwh = ds_all[var_name] / 3_600_000.0  # J/m² → kWh/m²
    
    # Round coordinates for numerical stability
    ghi_kwh['latitude'] = np.round(ghi_kwh['latitude'].values, 1)
    ghi_kwh['longitude'] = np.round(ghi_kwh['longitude'].values, 1)
    
    print(f"  ⚙️  Pivoting to wide format...")
    
    # Convert to DataFrame and pivot
    df = ghi_kwh.to_dataframe()
    df_pivot = df.pivot_table(index="time", columns=["latitude", "longitude"], values=var_name)
    df_pivot.index.name = "time"
    
    # Save to CSV
    df_pivot.to_csv(output_csv)
    
    print(f"  ✅ Saved solar incidence to {output_csv.name}")
    print(f"     Shape: {df_pivot.shape[0]:,} hours × {df_pivot.shape[1]:,} locations")
    print(f"     Date range: {df_pivot.index.min()} to {df_pivot.index.max()}")
    
    return df_pivot


def create_solar_ranking(solar_csv: Path, output_csv: Path) -> pd.DataFrame:
    """
    Rank locations by mean solar incidence (best locations first).
    
    Parameters
    ----------
    solar_csv : Path
        Path to solar incidence CSV
    output_csv : Path
        Output ranking CSV path
        
    Returns
    -------
    pd.DataFrame
        Ranking DataFrame with lat, lon, mean_solar_incidence, rank
    """
    print_subsection("Creating Solar Location Ranking")
    
    # Read header rows to get lat/lon
    with open(solar_csv, 'r') as f:
        reader = csv.reader(f)
        lat_row = next(reader)
        lon_row = next(reader)
    
    # Read data (skip headers)
    df = pd.read_csv(solar_csv, skiprows=3)
    df.rename(columns={df.columns[0]: "time"}, inplace=True)
    data_cols = [col for col in df.columns if col != "time"]
    
    # Compute mean per location
    mean_by_loc = df[data_cols].mean()
    
    # Extract coordinates
    lats = [float(x) for x in lat_row[1:]]
    lons = [float(x) for x in lon_row[1:]]
    
    # Create ranking DataFrame
    ranking_df = pd.DataFrame({
        'latitude': lats,
        'longitude': lons,
        'mean_solar_incidence_kwh_m2_per_hour': mean_by_loc.values
    })
    ranking_df = ranking_df.sort_values('mean_solar_incidence_kwh_m2_per_hour', 
                                         ascending=False).reset_index(drop=True)
    ranking_df['rank'] = ranking_df.index + 1
    
    # Save
    ranking_df.to_csv(output_csv, index=False)
    
    print(f"  ✅ Saved ranking to {output_csv.name}")
    print(f"\n  🏆 Top 5 Solar Locations:")
    for _, row in ranking_df.head(5).iterrows():
        print(f"     {int(row['rank'])}. ({row['latitude']:.1f}, {row['longitude']:.1f}) - "
              f"{row['mean_solar_incidence_kwh_m2_per_hour']:.4f} kWh/m²/h")
    
    return ranking_df


# Process solar data
solar_input = DUMPING_DIR / 'solar_incidence'
solar_output = IMPORT_DIR / 'solar_incidence_hourly_2024.csv'
solar_ranking_output = IMPORT_DIR / 'solar_incidence_ranking.csv'

print_section("1. SOLAR INCIDENCE DATA PROCESSING")

if solar_input.exists() and HAS_XARRAY:
    try:
        solar_df = process_solar_grib_to_csv(solar_input, solar_output)
        if solar_df is not None:
            solar_ranking = create_solar_ranking(solar_output, solar_ranking_output)
    except Exception as e:
        print(f"  ⚠️ Solar processing failed: {e}")
        print(f"     Attempting to load existing CSV...")
        if solar_output.exists():
            solar_df = pd.read_csv(solar_output, index_col=0, header=[0,1], parse_dates=True)
            print(f"  ✅ Loaded existing: {solar_df.shape}")
        else:
            solar_df = None
elif solar_output.exists():
    print_subsection("Loading Existing Solar Data")
    solar_df = pd.read_csv(solar_output, index_col=0, header=[0,1], parse_dates=True)
    print(f"  ✅ Loaded: {solar_df.shape[0]:,} hours × {solar_df.shape[1]:,} locations")
    if solar_ranking_output.exists():
        solar_ranking = pd.read_csv(solar_ranking_output)
        print(f"  ✅ Loaded ranking: {len(solar_ranking):,} locations")
else:
    print("  ❌ No solar data available. Download GRIB files to dumping/solar_incidence/")
    solar_df = None


### 1.2 Preprocessing Plots: Solar Analysis

Statistical analysis and visualization of solar incidence data.


In [ ]:
# ============================================================================
# 1.2 SOLAR: PREPROCESSING PLOTS & ANALYSIS
# ============================================================================

if 'solar_df' in globals() and solar_df is not None:
    print_section("1.2 SOLAR DATA ANALYSIS")
    
    # Statistics
    solar_values = solar_df.values.flatten()
    solar_values = solar_values[~np.isnan(solar_values)]
    solar_stats = compute_statistics(solar_values, "Solar")
    print_statistics(solar_stats, "Solar Incidence (kWh/m²/hour)")
    
    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # Monthly average
    monthly_avg = solar_df.mean(axis=1).resample('M').mean()
    axes[0, 0].bar(range(1, 13), monthly_avg.values, color='orange', alpha=0.7)
    axes[0, 0].set_title('Monthly Average Solar Incidence')
    axes[0, 0].set_xlabel('Month')
    axes[0, 0].set_ylabel('kWh/m²/hour')
    
    # Distribution
    axes[0, 1].hist(solar_values[solar_values > 0], bins=50, color='orange', alpha=0.7)
    axes[0, 1].set_title('Distribution (Non-Zero Values)')
    axes[0, 1].set_xlabel('kWh/m²/hour')
    
    # Sample day profile
    sample_day = solar_df.loc['2024-07-15':'2024-07-16'].mean(axis=1)
    axes[1, 0].plot(sample_day.index.hour, sample_day.values, 'o-', color='orange')
    axes[1, 0].set_title('Sample Day Profile (2024-07-15)')
    axes[1, 0].set_xlabel('Hour')
    
    # Annual variation
    daily_mean = solar_df.mean(axis=1).resample('D').mean()
    axes[1, 1].plot(daily_mean.index, daily_mean.values, linewidth=0.5, color='orange')
    axes[1, 1].set_title('Daily Mean Solar Incidence Over Year')
    axes[1, 1].set_xlabel('Date')
    
    plt.tight_layout()
    save_plot(fig, '01_solar_analysis.png')
    plt.show()
    print("✅ Solar analysis complete")
else:
    print("⚠️ Solar data not available")


## 2. Wind Incidence Data

**Source**: ERA5 reanalysis GRIB files  
**Input**: `dumping/wind_incidence/*.grib`  
**Output**: `import/wind_incidence_hourly_2024.csv`, `import/wind_incidence_ranking.csv`  
**Processing**: Extract u10/v10 components, calculate wind speed, pivot to hourly × locations format


### 2.1 Data Treatment: Wind GRIB to CSV


In [ ]:
# ============================================================================
# 2.1 WIND: GRIB TO CSV CONVERSION
# ============================================================================

def process_wind_grib_to_csv(input_dir: Path, output_csv: Path) -> Optional[pd.DataFrame]:
    """Process wind GRIB files: extract u10, v10 and compute wind speed."""
    if not HAS_XARRAY:
        print("  ⚠️ xarray not available")
        return None
    
    print_subsection("Processing Wind GRIB Files")
    
    grib_files = sorted([p for p in input_dir.rglob("*") 
                         if p.is_file() and not p.name.endswith('.idx')])
    
    if not grib_files:
        print(f"  ❌ No GRIB files found")
        return None
    
    print(f"  📂 Found {len(grib_files)} GRIB files")
    
    all_dfs = []
    for f in grib_files:
        try:
            ds = xr.open_dataset(f, engine="cfgrib")
            wind_speed = np.sqrt(ds["u10"]**2 + ds["v10"]**2)
            if "valid_time" in ds:
                wind_speed = wind_speed.assign_coords(time=ds["valid_time"])
            df = wind_speed.to_dataframe(name="wind_speed").reset_index()
            all_dfs.append(df)
            print(f"     ✓ Loaded {f.name}")
        except Exception as e:
            print(f"     ⚠️ Failed {f.name}: {e}")
    
    if not all_dfs:
        return None
    
    # Concatenate
    wind_data = pd.concat(all_dfs, ignore_index=True)
    wind_data = wind_data.rename(columns={"valid_time": "datetime"})
    wind_data = wind_data.dropna(subset=["wind_speed", "datetime"])
    
    # Round coordinates
    wind_data["latitude"] = wind_data["latitude"].round(1)
    wind_data["longitude"] = wind_data["longitude"].round(1)
    
    # Pivot
    wind_pivot = wind_data.pivot_table(
        index="datetime", 
        columns=["latitude", "longitude"], 
        values="wind_speed"
    )
    wind_pivot.index.name = "datetime"
    
    # Save
    wind_pivot.to_csv(output_csv)
    print(f"  ✅ Saved to {output_csv.name}")
    print(f"     Shape: {wind_pivot.shape[0]:,} × {wind_pivot.shape[1]:,}")
    
    return wind_pivot

def create_wind_ranking(wind_csv: Path, output_csv: Path) -> pd.DataFrame:
    """Rank wind locations by mean speed."""
    print_subsection("Creating Wind Location Ranking")
    
    df = pd.read_csv(wind_csv, index_col=0, header=[0,1], parse_dates=True)
    mean_by_loc = df.mean()
    
    lats = [round(float(col[0]), 1) for col in df.columns]
    lons = [round(float(col[1]), 1) for col in df.columns]
    
    ranking_df = pd.DataFrame({
        'latitude': lats,
        'longitude': lons,
        'mean_wind_speed_m_per_s': mean_by_loc.values
    })
    ranking_df = ranking_df.sort_values('mean_wind_speed_m_per_s', ascending=False).reset_index(drop=True)
    ranking_df['rank'] = ranking_df.index + 1
    ranking_df.to_csv(output_csv, index=False)
    
    print(f"  ✅ Saved ranking")
    print(f"\n  🏆 Top 5 Wind Locations:")
    for _, row in ranking_df.head(5).iterrows():
        print(f"     {int(row['rank'])}. ({row['latitude']:.1f}, {row['longitude']:.1f}) - "
              f"{row['mean_wind_speed_m_per_s']:.2f} m/s")
    
    return ranking_df

# Process wind data
wind_input = DUMPING_DIR / 'wind_incidence'
wind_output = IMPORT_DIR / 'wind_incidence_hourly_2024.csv'
wind_ranking_output = IMPORT_DIR / 'wind_incidence_ranking.csv'

print_section("2. WIND INCIDENCE DATA PROCESSING")

if wind_input.exists() and HAS_XARRAY:
    try:
        wind_df = process_wind_grib_to_csv(wind_input, wind_output)
        if wind_df is not None:
            wind_ranking = create_wind_ranking(wind_output, wind_ranking_output)
    except Exception as e:
        print(f"  ⚠️ Wind processing failed: {e}")
        if wind_output.exists():
            wind_df = pd.read_csv(wind_output, index_col=0, header=[0,1], parse_dates=True)
            print(f"  ✅ Loaded existing")
        else:
            wind_df = None
elif wind_output.exists():
    print_subsection("Loading Existing Wind Data")
    wind_df = pd.read_csv(wind_output, index_col=0, header=[0,1], parse_dates=True)
    print(f"  ✅ Loaded: {wind_df.shape}")
else:
    print("  ❌ No wind data available")
    wind_df = None


### 2.2 Preprocessing Plots: Wind Analysis


In [ ]:
# ============================================================================
# 2.2 WIND: PREPROCESSING PLOTS
# ============================================================================

if 'wind_df' in globals() and wind_df is not None:
    print_section("2.2 WIND DATA ANALYSIS")
    
    wind_values = wind_df.values.flatten()
    wind_values = wind_values[~np.isnan(wind_values)]
    wind_stats = compute_statistics(wind_values, "Wind")
    print_statistics(wind_stats, "Wind Speed (m/s)")
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # Monthly average
    monthly_avg = wind_df.mean(axis=1).resample('M').mean()
    axes[0, 0].bar(range(1, 13), monthly_avg.values, color='steelblue', alpha=0.7)
    axes[0, 0].set_title('Monthly Average Wind Speed')
    axes[0, 0].set_xlabel('Month')
    axes[0, 0].set_ylabel('m/s')
    
    # Distribution
    axes[0, 1].hist(wind_values, bins=50, color='steelblue', alpha=0.7)
    axes[0, 1].set_title('Wind Speed Distribution')
    axes[0, 1].set_xlabel('m/s')
    
    # Sample day
    sample_day = wind_df.loc['2024-03-15':'2024-03-16'].mean(axis=1)
    axes[1, 0].plot(sample_day.index.hour, sample_day.values, 'o-', color='steelblue')
    axes[1, 0].set_title('Sample Day Profile (2024-03-15)')
    axes[1, 0].set_xlabel('Hour')
    axes[1, 0].set_ylabel('m/s')
    
    # Annual variation
    daily_mean = wind_df.mean(axis=1).resample('D').mean()
    axes[1, 1].plot(daily_mean.index, daily_mean.values, linewidth=0.5, color='steelblue')
    axes[1, 1].set_title('Daily Mean Wind Speed')
    axes[1, 1].set_xlabel('Date')
    
    plt.tight_layout()
    save_plot(fig, '02_wind_analysis.png')
    plt.show()
    print("✅ Wind analysis complete")
else:
    print("⚠️ Wind data not available")


## 3. Spot Prices & Exchange Rates

**Sources**: ENTSO-E (prices), ECB (EUR/CHF), Dukascopy (USD/CHF)  
**Input**: `dumping/energy_price_all_2024.json`, `dumping/chf_to_eur_2024.csv`, `dumping/DAT_ASCII_USDCHF_M1_2024.csv`  
**Output**: `import/spot_price_hourly.csv`, `import/chf_to_eur_2024.csv`  
**Processing**: Convert prices to CHF/MWh, handle timezones


### 3.1 Data Treatment: Spot Prices


In [ ]:
# ============================================================================
# 3.1 SPOT PRICES: JSON TO CSV
# ============================================================================

print_section("3. SPOT PRICES & EXCHANGE RATES")

# Process spot prices
price_input = DUMPING_DIR / 'energy_price_all_2024.json'
price_output = IMPORT_DIR / 'spot_price_hourly.csv'

if price_input.exists():
    print_subsection("Processing Spot Prices")
    
    with open(price_input, 'r') as f:
        price_data = json.load(f)
    
    # Convert to DataFrame
    price_df = pd.DataFrame([
        {'time': pd.to_datetime(entry['time']), 'price': float(entry['price'])}
        for entry in price_data
    ])
    price_df = price_df.set_index('time').sort_index()
    
    # Save
    price_df.to_csv(price_output)
    print(f"  ✅ Saved {len(price_df):,} hourly price points")
    print(f"     Range: {price_df.index.min()} to {price_df.index.max()}")
    print(f"     Price range: {price_df['price'].min():.2f} - {price_df['price'].max():.2f} CHF/MWh")
    
elif price_output.exists():
    price_df = pd.read_csv(price_output, index_col=0, parse_dates=True)
    print(f"  ✅ Loaded existing prices: {len(price_df):,} points")
else:
    print("  ❌ No price data available")
    price_df = None

# Process exchange rates
exchange_input = DUMPING_DIR / 'chf_to_eur_2024.csv'
exchange_output = IMPORT_DIR / 'chf_to_eur_2024.csv'

if exchange_input.exists():
    print_subsection("Processing Exchange Rates")
    
    exchange_df = pd.read_csv(exchange_input)
    exchange_df.to_csv(exchange_output, index=False)
    print(f"  ✅ Saved exchange rates: {len(exchange_df)} days")
    
    if 'CHF to EUR' in exchange_df.columns:
        rate_col = 'CHF to EUR'
        print(f"     Mean rate: {exchange_df[rate_col].mean():.4f}")
        print(f"     Range: {exchange_df[rate_col].min():.4f} - {exchange_df[rate_col].max():.4f}")
        
elif exchange_output.exists():
    exchange_df = pd.read_csv(exchange_output)
    print(f"  ✅ Loaded existing exchange rates")
else:
    print("  ❌ No exchange rate data")
    exchange_df = None


### 3.2 Preprocessing Plots: Price Analysis


In [ ]:
# ============================================================================
# 3.2 SPOT PRICES: ANALYSIS
# ============================================================================

if 'price_df' in globals() and price_df is not None:
    print_section("3.2 SPOT PRICE ANALYSIS")
    
    price_values = price_df['price'].values
    price_stats = compute_statistics(price_values, "Price")
    print_statistics(price_stats, "Spot Prices (CHF/MWh)")
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # Time series
    axes[0, 0].plot(price_df.index, price_df['price'], linewidth=0.5, color='darkgreen')
    axes[0, 0].set_title('Hourly Spot Prices (2024)')
    axes[0, 0].set_ylabel('CHF/MWh')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Distribution
    axes[0, 1].hist(price_values, bins=50, color='darkgreen', alpha=0.7)
    axes[0, 1].set_title('Price Distribution')
    axes[0, 1].set_xlabel('CHF/MWh')
    axes[0, 1].axvline(price_stats['mean'], color='red', linestyle='--', label='Mean')
    axes[0, 1].legend()
    
    # Monthly average
    monthly_avg = price_df.resample('M').mean()
    axes[1, 0].bar(range(1, len(monthly_avg) + 1), monthly_avg['price'].values, 
                   color='darkgreen', alpha=0.7)
    axes[1, 0].set_title('Monthly Average Prices')
    axes[1, 0].set_xlabel('Month')
    axes[1, 0].set_ylabel('CHF/MWh')
    
    # Daily profile
    price_df['hour'] = price_df.index.hour
    hourly_avg = price_df.groupby('hour')['price'].mean()
    axes[1, 1].plot(hourly_avg.index, hourly_avg.values, 'o-', color='darkgreen', linewidth=2)
    axes[1, 1].set_title('Average Daily Price Profile')
    axes[1, 1].set_xlabel('Hour of Day')
    axes[1, 1].set_ylabel('CHF/MWh')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    save_plot(fig, '03_spot_price_analysis.png')
    plt.show()
    print("✅ Price analysis complete")
else:
    print("⚠️ Price data not available")


## 4. Demand & Water Data

**Sources**: ENTSO-E (demand), BFE (hydro), Open-Meteo (water inflow)  
**Input**: `dumping/monthly_hourly_load_values_2024.csv`, `dumping/water_monthly_ror_2024.csv`, `dumping/Swiss_Water_Hourly_2024.csv`  
**Output**: `import/monthly_hourly_load_values_2024.csv`, `import/water_quarterly_ror_2024.csv`  
**Processing**: Load demand, disaggregate monthly RoR to 15-min intervals via cubic spline


In [ ]:
# ============================================================================
# 4. DEMAND & WATER DATA
# ============================================================================

print_section("4. DEMAND & WATER DATA PROCESSING")

# Demand data
demand_input = DUMPING_DIR / 'monthly_hourly_load_values_2024.csv'
demand_output = IMPORT_DIR / 'monthly_hourly_load_values_2024.csv'

if demand_input.exists():
    print_subsection("Processing Demand Data")
    demand_df = pd.read_csv(demand_input)
    demand_df.to_csv(demand_output, index=False)
    print(f"  ✅ Saved demand data: {len(demand_df):,} entries")
    
    if 'Value' in demand_df.columns:
        print(f"     Mean demand: {demand_df['Value'].mean():.2f} MW")
        print(f"     Peak demand: {demand_df['Value'].max():.2f} MW")
elif demand_output.exists():
    demand_df = pd.read_csv(demand_output)
    print(f"  ✅ Loaded demand: {len(demand_df):,} entries")
else:
    print("  ❌ No demand data")
    demand_df = None

# Water RoR data - disaggregate monthly to 15-min
ror_input = DUMPING_DIR / 'water_monthly_ror_2024.csv'
ror_output = IMPORT_DIR / 'water_quarterly_ror_2024.csv'

if ror_input.exists():
    print_subsection("Processing Run-of-River Data")
    
    monthly_ror = pd.read_csv(ror_input)
    monthly_ror['Month'] = pd.to_datetime(monthly_ror['Month'])
    monthly_ror = monthly_ror.set_index('Month')
    
    # Convert GWh to MW
    monthly_ror['hours_in_month'] = monthly_ror.index.days_in_month * 24
    monthly_ror['avg_power_MW'] = (monthly_ror['RoR_GWh'] / monthly_ror['hours_in_month']) * 1000
    
    # Create interpolation points
    start_of_year = pd.Timestamp('2024-01-01')
    month_midpoints = []
    power_values = []
    
    for idx, row in monthly_ror.iterrows():
        mid_day = idx + pd.Timedelta(days=idx.days_in_month/2)
        month_midpoints.append(mid_day)
        power_values.append(row['avg_power_MW'])
    
    x_days = np.array([(d - start_of_year).total_seconds() / 86400 for d in month_midpoints])
    y_power = np.array(power_values)
    
    # Cubic spline interpolation
    x_extended = np.concatenate([[x_days[0] - 365], x_days, [x_days[-1] + 365]])
    y_extended = np.concatenate([[y_power[-1]], y_power, [y_power[0]]])
    cs = CubicSpline(x_extended, y_extended, bc_type='natural')
    
    # 15-minute timestamps
    timestamps_15min = pd.date_range(start='2024-01-01 00:00:00', 
                                     end='2024-12-31 23:45:00', freq='15min')
    x_15min = np.array([(t - start_of_year).total_seconds() / 86400 for t in timestamps_15min])
    power_15min_MW = np.maximum(cs(x_15min), 0)
    
    # Save
    df_ror_15min = pd.DataFrame({
        'timestamp': timestamps_15min,
        'RoR_MW': power_15min_MW
    })
    df_ror_15min.to_csv(ror_output, index=False)
    
    print(f"  ✅ Saved 15-min RoR: {len(df_ror_15min):,} timesteps")
    print(f"     Power range: {power_15min_MW.min():.2f} - {power_15min_MW.max():.2f} MW")
    
    # Energy conservation check
    energy_15min = (power_15min_MW * 0.25 / 1000).sum()  # 15min = 0.25h
    energy_original = monthly_ror['RoR_GWh'].sum()
    print(f"     Energy check: {energy_original:.2f} GWh (original) vs {energy_15min:.2f} GWh (interpolated)")
    print(f"     Error: {abs(energy_15min - energy_original) / energy_original * 100:.3f}%")
    
elif ror_output.exists():
    df_ror_15min = pd.read_csv(ror_output)
    print(f"  ✅ Loaded RoR: {len(df_ror_15min):,} timesteps")
else:
    print("  ❌ No RoR data")
    df_ror_15min = None

# Water hourly data
water_input = DUMPING_DIR / 'Swiss_Water_Hourly_2024.csv'
water_output = IMPORT_DIR / 'Swiss_Water_Hourly_2024.csv'

if water_input.exists():
    print_subsection("Processing Water Inflow Data")
    water_df = pd.read_csv(water_input)
    water_df.to_csv(water_output, index=False)
    print(f"  ✅ Saved water data: {len(water_df):,} hours")
elif water_output.exists():
    water_df = pd.read_csv(water_output)
    print(f"  ✅ Loaded water: {len(water_df):,} hours")
else:
    print("  ❌ No water data")
    water_df = None


## 5. PPU Cost Analysis

**Source**: Cost table and analyze_ppu_chains.py logic  
**Input**: `dumping/cost_table_tidy.csv`, `dumping/ppu_constructs_components.csv`  
**Output**: `import/cost_table_tidy.csv`, `import/ppu_constructs_components.csv`, `import/ppu_efficiency_lcoe_analysis.csv`  
**Processing**: Copy cost tables, run PPU chain analysis for efficiency and LCOE


In [ ]:
# ============================================================================
# 5. PPU COST ANALYSIS
# ============================================================================

print_section("5. PPU COST ANALYSIS")

# Copy cost table
cost_input = DUMPING_DIR / 'cost_table_tidy.csv'
cost_output = IMPORT_DIR / 'cost_table_tidy.csv'

if cost_input.exists():
    print_subsection("Processing Cost Table")
    cost_df = pd.read_csv(cost_input)
    cost_df.to_csv(cost_output, index=False)
    print(f"  ✅ Saved cost table: {len(cost_df)} components")
elif cost_output.exists():
    cost_df = pd.read_csv(cost_output)
    print(f"  ✅ Loaded cost table: {len(cost_df)} components")
else:
    print("  ❌ No cost table")
    cost_df = None

# Copy PPU constructs
ppu_input = DUMPING_DIR / 'ppu_constructs_components.csv'
ppu_output = IMPORT_DIR / 'ppu_constructs_components.csv'

if ppu_input.exists():
    print_subsection("Processing PPU Constructs")
    ppu_df = pd.read_csv(ppu_input)
    ppu_df.to_csv(ppu_output, index=False)
    print(f"  ✅ Saved PPU constructs: {len(ppu_df)} PPUs")
elif ppu_output.exists():
    ppu_df = pd.read_csv(ppu_output)
    print(f"  ✅ Loaded PPU constructs: {len(ppu_df)} PPUs")
else:
    print("  ❌ No PPU constructs")
    ppu_df = None

# Run PPU efficiency analysis (simplified version)
if cost_df is not None and ppu_df is not None:
    print_subsection("Running PPU Efficiency Analysis")
    
    # This is a simplified placeholder - in practice, run analyze_ppu_chains.py
    print("  ⚙️  PPU efficiency analysis should be run separately using analyze_ppu_chains.py")
    print("     This script computes LCOE and efficiency for all PPU chains")
    
    # Check if analysis already exists
    ppu_analysis_output = IMPORT_DIR / 'ppu_efficiency_lcoe_analysis.csv'
    if ppu_analysis_output.exists():
        ppu_analysis = pd.read_csv(ppu_analysis_output)
        print(f"  ✅ Existing analysis found: {len(ppu_analysis)} PPUs")
        print(f"\n  📊 PPU Summary:")
        for _, row in ppu_analysis.head(5).iterrows():
            print(f"     {row['PPU']:<15} Eff: {row['Efficiency']*100:5.1f}%  LCOE: {row['LCOE_CHF_kWh']:.4f} CHF/kWh")
    else:
        print(f"  ⚠️  Run analyze_ppu_chains.py to generate {ppu_analysis_output.name}")

print("\n✅ PPU cost data processing complete")


## 6. Final Validation

Check all output files are ready for `Energy_Portfolio_Optimization.ipynb`.


In [ ]:
# ============================================================================
# 6. FINAL VALIDATION
# ============================================================================

print_section("6. FINAL VALIDATION", char="=", width=80)

# Required output files
required_files = [
    ('solar_incidence_hourly_2024.csv', 'Solar GHI (hourly × locations)'),
    ('wind_incidence_hourly_2024.csv', 'Wind speed (hourly × locations)'),
    ('solar_incidence_ranking.csv', 'Solar location ranking'),
    ('wind_incidence_ranking.csv', 'Wind location ranking'),
    ('spot_price_hourly.csv', 'Spot electricity prices (CHF/MWh)'),
    ('chf_to_eur_2024.csv', 'EUR/CHF exchange rates'),
    ('monthly_hourly_load_values_2024.csv', 'Electricity demand'),
    ('water_quarterly_ror_2024.csv', 'Run-of-river (15-min)'),
    ('Swiss_Water_Hourly_2024.csv', 'Water inflow (hourly)'),
    ('cost_table_tidy.csv', 'Component cost table'),
    ('ppu_constructs_components.csv', 'PPU definitions'),
    ('ppu_efficiency_lcoe_analysis.csv', 'PPU efficiency & LCOE'),
]

print("\n📋 Checking Required Output Files:\n")

all_ok = True
ready_files = []
missing_files = []

for filename, description in required_files:
    path = IMPORT_DIR / filename
    if path.exists():
        size_mb = path.stat().st_size / (1024 * 1024)
        status = f"✅ {filename:<45} {size_mb:>7.2f} MB"
        print(status)
        ready_files.append(filename)
    else:
        status = f"❌ {filename:<45} MISSING"
        print(status)
        missing_files.append(filename)
        all_ok = False

print("\n" + "="*80)

if all_ok:
    print("🎉 ALL FILES READY!")
    print(f"   {len(ready_files)} / {len(required_files)} files present in import/")
    print("\n   ✅ You can now run Energy_Portfolio_Optimization.ipynb")
else:
    print(f"⚠️  {len(missing_files)} / {len(required_files)} FILES MISSING")
    print(f"\n   Missing files:")
    for f in missing_files:
        print(f"     - {f}")
    print(f"\n   💡 Check the dumping/ folder for required raw data")
    print(f"      and re-run relevant sections above.")

print("="*80)

# Summary statistics
print("\n📊 PIPELINE SUMMARY:")
print(f"   Dumping directory:  {DUMPING_DIR.absolute()}")
print(f"   Import directory:   {IMPORT_DIR.absolute()}")
print(f"   Plot directory:     {PLOT_DIR.absolute()}")
print(f"   Files ready:        {len(ready_files)} / {len(required_files)}")
print(f"   Year processed:     {YEAR}")
print(f"   Hours in year:      {N_HOURS_YEAR}")

print("\n✅ Data preprocessing pipeline complete!")


## Appendix: Data Sources

Complete list of raw data sources and download instructions.

### Data Sources Reference

| Dataset | Source | URL | File Format |
|---------|--------|-----|-------------|
| **Solar** | Open-Meteo ERA5 | https://open-meteo.com/en/docs/historical-weather-api | GRIB |
| **Wind** | Open-Meteo ERA5 | https://open-meteo.com/en/docs/historical-weather-api | GRIB |
| **Spot Prices** | ENTSO-E Transparency Platform | https://transparency.entsoe.eu/ | JSON |
| **EUR/CHF** | European Central Bank | https://www.ecb.europa.eu/ | CSV |
| **USD/CHF** | Dukascopy | https://www.dukascopy.com/ | CSV |
| **Demand** | ENTSO-E | https://transparency.entsoe.eu/ | CSV |
| **RoR Hydro** | Swiss Federal Office of Energy (BFE) | https://www.bfe.admin.ch/ | CSV |
| **Water Inflow** | Open-Meteo Precipitation | https://open-meteo.com/ | CSV |
| **Palm Oil** | REA Holdings | https://www.rea.co.uk/ | XLS/CSV |
| **Cost Table** | Project Analysis | Manual compilation | CSV |

### Architecture Benefits

This modular design provides:

1. **Separation of Concerns**: Raw data (dumping/) vs. processed data (import/)
2. **Reproducibility**: Each dataset has dedicated treatment and validation
3. **Maintainability**: Functions are reusable and well-documented
4. **Traceability**: Each step logs what it does with clear status messages
5. **Visualization**: Every dataset includes preprocessing plots for validation
6. **Software Engineering**: Type hints, docstrings, error handling, DRY principles

### Usage Notes

- **First Run**: Ensure all raw data files are in `dumping/` folder before running
- **Subsequent Runs**: Notebook intelligently loads existing processed files
- **Debugging**: Each section is independent - can be run individually
- **Extensibility**: Easy to add new data sources following the same pattern

---

**Version**: 1.0  
**Compatible with**: Energy_Portfolio_Optimization.ipynb  
**Python**: 3.9+  
**Key Dependencies**: pandas, numpy, scipy, matplotlib, xarray, cfgrib


### 1.2 Preprocessing Plots: Solar Analysis


In [ ]:
# ============================================================================
# 1.2 SOLAR: PREPROCESSING PLOTS & ANALYSIS
# ============================================================================

if 'solar_df' in globals() and solar_df is not None and len(solar_df) > 0:
    print_section("1.2 SOLAR DATA ANALYSIS & VISUALIZATION")
    
    # Flatten to 1D for statistics
    solar_values = solar_df.values.flatten()
    solar_values = solar_values[~np.isnan(solar_values)]
    
    # Compute and print statistics
    solar_stats = compute_statistics(solar_values, "Solar")
    print_statistics(solar_stats, "Solar Incidence Statistics (All Locations, All Hours)")
    
    # Annual total per location
    annual_per_loc = solar_df.sum(axis=0)
    print(f"\n📊 Annual Totals per Location:")
    print(f"  Best location:  {annual_per_loc.max():.1f} kWh/m²/year")
    print(f"  Worst location: {annual_per_loc.min():.1f} kWh/m²/year")
    print(f"  Mean:           {annual_per_loc.mean():.1f} kWh/m²/year")
    
    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # 1. Monthly average
    monthly_avg = solar_df.mean(axis=1).resample('M').mean()
    axes[0, 0].bar(monthly_avg.index.month, monthly_avg.values, 
                   color='orange', alpha=0.7, edgecolor='black')
    axes[0, 0].set_xlabel('Month')
    axes[0, 0].set_ylabel('Mean Solar Incidence (kWh/m²/hour)')
    axes[0, 0].set_title('Monthly Average Solar Incidence (2024)')
    axes[0, 0].set_xticks(range(1, 13))
    axes[0, 0].set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                                'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    axes[0, 0].grid(True, alpha=0.3, axis='y')
    
    # 2. Distribution histogram
    axes[0, 1].hist(solar_values[solar_values > 0], bins=50, 
                    color='orange', alpha=0.7, edgecolor='black')
    axes[0, 1].set_xlabel('Solar Incidence (kWh/m²/hour)')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title('Distribution of Non-Zero Solar Incidence')
    axes[0, 1].grid(True, alpha=0.3, axis='y')
    
    # 3. Random day profile
    random_date = pd.Timestamp('2024-07-15')  # Mid-summer day
    if random_date in solar_df.index:
        day_profile = solar_df.loc[random_date:random_date + pd.Timedelta(days=1)].mean(axis=1)
        axes[1, 0].plot(day_profile.index.hour, day_profile.values, 
                       'o-', linewidth=2, markersize=6, color='orange')
        axes[1, 0].fill_between(day_profile.index.hour, day_profile.values, alpha=0.3, color='orange')
        axes[1, 0].set_xlabel('Hour of Day')
        axes[1, 0].set_ylabel('Average Solar Incidence (kWh/m²/hour)')
        axes[1, 0].set_title(f'Daily Profile - {random_date.strftime("%Y-%m-%d")} (Average Across Locations)')
        axes[1, 0].set_xlim(0, 23)
        axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Annual heatmap (monthly x location percentiles)
    monthly_data = []
    for month in range(1, 13):
        month_data = solar_df[solar_df.index.month == month].mean(axis=0)
        monthly_data.append([
            month_data.quantile(0.1),
            month_data.quantile(0.25),
            month_data.quantile(0.5),
            month_data.quantile(0.75),
            month_data.quantile(0.9)
        ])
    
    heatmap_data = np.array(monthly_data).T
    im = axes[1, 1].imshow(heatmap_data, aspect='auto', cmap='YlOrRd', origin='lower')
    axes[1, 1].set_xticks(range(12))
    axes[1, 1].set_xticklabels(['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D'])
    axes[1, 1].set_yticks(range(5))
    axes[1, 1].set_yticklabels(['P10', 'P25', 'P50', 'P75', 'P90'])
    axes[1, 1].set_xlabel('Month')
    axes[1, 1].set_ylabel('Location Percentile')
    axes[1, 1].set_title('Solar Incidence: Monthly Variation by Location Quality')
    plt.colorbar(im, ax=axes[1, 1], label='kWh/m²/hour')
    
    plt.tight_layout()
    save_plot(fig, '01_solar_analysis.png')
    plt.show()
    
    print("\n✅ Solar data processing complete!")
    
else:
    print("\n⚠️ Solar data not available - skipping analysis")
